In [ ]:
import pandas as pd 
import numpy as np

file_path = '../data/imdb_top_1000.csv'

try:
    data_frame = pd.read_csv(file_path)
    print("Dados carregados! As 5 primeiras linhas são: ")
    display(data_frame.head())
except FileNotFoundError: 
    print("Falha!!")


print("informações gerais: ")
data_frame.info()

print("Estatística das tabelas: ")
display(data_frame.describe())

data_frame['Runtime'] = data_frame['Runtime'].str.replace(' min', '').astype(int)
print("Coluna Runtime limpa e convertida para inteiros")

data_frame['Gross'] = data_frame['Gross'].str.replace(',', '', regex=False)
data_frame['Gross'] = pd.to_numeric(data_frame['Gross'], errors='coerce')
print("Coluna Gross limpa e convertida para float.")

mode_certificate = data_frame['Certificate'].mode()[0]
data_frame['Certificate'].fillna(mode_certificate, inplace=True)
print(f"Valores nulos de Certificate preenchidos com a moda: '{mode_certificate}'")

mediana_metascore = data_frame['Meta_score'].median()
data_frame['Meta_score'].fillna(mediana_metascore, inplace=True)
print(f"Valores nulos de 'Meta_score' preenchidos com a mediana: {mediana_metascore}")

print("\nVerificando as informações do DataFrame depois da limpeza:")
data_frame.info()

print("Resumo estatístico após a limpeza:")
display(data_frame.describe())


Estatística das tabelas: 


,Unnamed: 0,IMDB_Rating,Meta_score,No_of_Votes
count,999.000000,999.000000,842.000000,9.990000e+02
mean,500.000000,7.947948,77.969121,2.716214e+05
std,288.530761,0.272290,12.383257,3.209126e+05
min,1.000000,7.600000,28.000000,2.508800e+04
25%,250.500000,7.700000,70.000000,5.547150e+04
50%,500.000000,7.900000,79.000000,1.383560e+05
75%,749.500000,8.100000,87.000000,3.731675e+05
max,999.000000,9.200000,100.000000,2.303232e+06
